In [1]:
# This chunk of code make it possible to use src functions
import sys
import os
module_path = os.path.abspath(os.path.join('../src'))

if module_path not in sys.path:
    sys.path.append(module_path)
import warnings
warnings.filterwarnings('ignore')

from dsproject import dsproject
from preprocess import *

dsp = dsproject()

In [2]:
'''
min_date_text = '20170201'
max_date_text = '20170501'
'''
min_date_text = '20160401'
max_date_text = '20160701'
min_date = datetime.strptime(min_date_text, '%Y%m%d')
max_date = datetime.strptime(max_date_text, '%Y%m%d')

In [3]:
df0 = dsp.read_data('EDW_SOR_CC_AR', nrows=10)

In [4]:
df = pd.DataFrame()
j=0
for chunk in dsp.read_data('EDW_SOR_CC_AR', usecols=[3], chunksize=1000000):
    j=j+1
    #chunk.columns = df0.columns
    chunk.columns = ['POSN_DT']
    chunk['POSN_DT'] = parse_dates(chunk['POSN_DT'], '%Y-%m-%d')
    df_train = filter_dates_1(chunk, min_date, max_date, date_column='POSN_DT')
    if not df_train.empty:
        if df.empty:
            df = df_train
        else:
            df = pd.concat([df, df_train])
        print('found')
    else:
        print(j)

1
2
found
found
5
6
7
found
found
found
found
12
13


In [5]:
print (len(df.index))
print (df['POSN_DT'].min())
print (df['POSN_DT'].max())

1640920
2016-04-30 00:00:00
2016-06-30 00:00:00


In [6]:
skip=list(set(range(0, list(chunk.index)[-1]))-set(list(df.index)))

In [8]:
prep = dsp.read_data('EDW_SOR_CC_AR', skiprows=skip)
# print (list(prep.columns))
# print (prep['POSN_DT'].min())
# print (prep['POSN_DT'].max())
print (len(prep.index))

1640921


In [10]:
dsp.write_table(prep, 'sor_cc_ar_filtered_' + min_date_text + '_' + max_date_text, 'preprocess')